# Environment setting

In [4]:
import codecs
import json

from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
import time

import ast
import jieba
from gensim.models import word2vec
from gensim.models import FastText
from glove import Glove, Corpus

warnings.filterwarnings("ignore")
%matplotlib

stopwords = [line.strip() for line in codecs.open(
    r'./stopwords.txt', 'r', 'utf-8').readlines()]

Using matplotlib backend: Qt5Agg


In [5]:
text = pd.read_csv(r'../../Data/TRAINSET_NEWS.csv')
stock = pd.read_csv(r'../../Data/TRAINSET_STOCK.csv')

# Prepare target vector

In [6]:
stock.head()

,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb,y
0,801010,20140401,农林牧渔,1668.75,1668.54,1689.12,1689.07,22.13,1.33,34914.0,291113.0,41.51,2.77,1
1,801010,20140402,农林牧渔,1688.72,1684.53,1693.41,1692.24,3.17,0.19,36300.0,289020.0,41.63,2.79,1
2,801010,20140403,农林牧渔,1693.05,1679.85,1697.73,1685.71,-6.53,-0.39,31403.0,259464.0,41.38,2.78,0
3,801010,20140404,农林牧渔,1681.92,1680.34,1698.44,1698.25,12.54,0.74,28648.0,240940.0,41.76,2.80,1
4,801010,20140408,农林牧渔,1693.24,1692.22,1706.84,1706.84,8.59,0.51,35012.0,312423.0,42.00,2.79,1


In [7]:
target_df = (stock.loc[(stock['y'] == 1)]).groupby(['trade_date', 'ts_code'])[
    'y'].count().unstack().fillna(0).astype(int)

# Load word embedding

## Load word2vec

In [8]:
w2vmodel = word2vec.Word2Vec.load("./Word_Embedding_Model/w2v.model")

## Load GloVe

In [9]:
corpus_model = Corpus.load('./Word_Embedding_Model/glov_corpus.model')
glove = Glove.load('./Word_Embedding_Model/glove.model')

## Load Fasttext

In [10]:
fttmodel = FastText.load(r'./Word_Embedding_Model/fasttext.model')

## Concatenate vector

In [11]:
glove.word_vectors.shape, len(fttmodel.wv.vocab), len(w2vmodel.wv.vocab)

((126565, 100), 126565, 126565)

In [34]:
# word_list = list(w2vmodel.wv.vocab.keys())
vector_size = 100
word_index = glove.dictionary
nb_words = len(word_index)

count = 0.5
embedding_word2vec_matrix = np.zeros((nb_words + 1, vector_size))
for word, i in tqdm(word_index.items()):
    embedding_vector = w2vmodel[word] if word in w2vmodel else None
    if embedding_vector is not None:
        count += 1
        embedding_word2vec_matrix[i] = embedding_vector
    else:
        unk_vec = np.random.random(victor_size) * 0.5
        unk_vec = unk_vec - unk_vec.mean()
        embedding_word2vec_matrix[i] = unk_vec


glove_count = 0
embedding_glove_matrix = np.zeros((nb_words + 1, vector_size))
for word, i in tqdm(word_index.items()):
    embedding_glove_vector = glove.word_vectors[glove.dictionary[word]
                                                ] if word in glove.dictionary else None
    if embedding_glove_vector is not None:
        glove_count += 1
        embedding_glove_matrix[i] = embedding_glove_vector
    else:
        unk_vec = np.random.random(victor_size) * 0.5
        unk_vec = unk_vec - unk_vec.mean()
        embedding_glove_matrix[i] = unk_vec


ftt_count = 0
embedding_ftt_matrix = np.zeros((nb_words + 1, vector_size))
for word, i in tqdm(word_index.items()):
    embedding_ftt_vector = fttmodel[word] if word in fttmodel else None
    if embedding_ftt_vector is not None:
        ftt_count += 1
        embedding_ftt_matrix[i] = embedding_ftt_vector
    else:
        unk_vec = np.random.random(victor_size) * 0.5
        unk_vec = unk_vec - unk_vec.mean()
        embedding_ftt_matrix[i] = unk_vec

embedding_matrix = np.concatenate(
    (embedding_word2vec_matrix, embedding_glove_matrix, embedding_ftt_matrix), axis=1)
print(embedding_matrix.shape, count * 1.0 /
      embedding_matrix.shape[0], glove_count*1.0/embedding_matrix.shape[0], ftt_count*1.0/embedding_matrix.shape[0])

100%|██████████| 126565/126565 [00:02<00:00, 45001.73it/s]


(126566, 300) 0.9999960494919646 0.9999920989839294 0.9999920989839294


# Load word segment

In [13]:
seg_text = pd.read_csv(r'./Word_Embedding_Model/seg_words.csv')
seg_text.head()

,date,title,content
0,20140414,"['习近平', '空军', '机关', '调研', '时', '强调', '加快', '建设...","['中共中央', '总书记', '国家', '主席', '中央军委', '主席', '习近平..."
1,20140414,"['利比亚', '临时政府', '总理', '辞职']","['本月', '8', '号', '刚刚', '正式', '任命', '利比亚', '临时政..."
2,20140414,"['关注', '乌克兰', '局势']","['代行', '乌克兰', '总统', '职责', '乌克兰', '议长', '图尔', '..."
3,20140414,"['国内', '联播', '快讯']","['低碳', '中国', '行', '正式', '启动', '国家', '发展', '改革'..."
4,20140414,"['刘汉', '36', '人涉', '黑案', '继续', '开庭审理']","['刘汉', '刘维', '36', '人', '涉嫌', '犯', '组织', '领导',..."


In [14]:
def merge_func(x):
    x = ast.literal_eval(x)
    r_list = []
    for i in x:
        r_list += i
    return r_list

## Merge news title from the same date

In [15]:
join_title = seg_text[['date', 'title']]
join_title = join_title.groupby(
    ['date'])['title'].apply(','.join).reset_index()
join_title['title'] = join_title['title'].apply(merge_func)

## Merge news content from the same date

In [16]:
join_content = seg_text[['date', 'content']]
join_content = join_content.groupby(
    ['date'])['content'].apply(','.join).reset_index()
join_content['content'] = join_content['content'].apply(merge_func)

# Contact title and content

In [17]:
joined_text = join_title.merge(join_content, on='date')
joined_text.head()

,date,title,content
0,20140414,"[习近平, 空军, 机关, 调研, 时, 强调, 加快, 建设, 一支, 空天, 一体, 攻...","[中共中央, 总书记, 国家, 主席, 中央军委, 主席, 习近平, 14, 日, 专程到,..."
1,20140415,"[医生, 贾永青, 传递, 爱, 感动, 国际, 联播, 快讯, 搜寻, MH370, 航班...","[几天, 我台, 走, 基层, 节目, 连续, 报道, 河北, 定州, 人民, 医院, 32..."
2,20140416,"[国际, 联播, 快讯, 关注, 乌克兰, 局势, 乌, 军队, 东部, 地区, 开展, 强...","[约旦, 驻, 利比亚, 大使, 遭绑架, 利比亚, 外交部, 15, 号, 证实, 约旦,..."
3,20140417,"[凡人, 善举, 广西, 市民, 见义勇为, 巧施, 妙计, 擒, 劫匪, 国际, 联播, ...","[前两天, 广西北海, 一位, 市民, 目睹, 一起, 抢夺案, 后, 没有, 选择, 离开..."
4,20140418,"[国际, 联播, 快讯, 俄罗斯, 总统, 俄, 民众, 直接对话, 普京, 乌, 境内, ...","[伊朗, 举行, 建军节, 阅兵式, 18, 号, 伊朗, 首都, 德黑兰, 南郊, 霍梅尼..."


# Padding size

In [26]:
title_size = int(np.percentile(joined_text['title'].str.len(), 95))
content_size = int(np.percentile(joined_text['content'].str.len(), 95))

# Replace word embedding

In [35]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


ImportError: cannot import name 'abs'

# Embedding

In [ ]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim


class GRU(nn.Module):
    def __init__(self, weights_matrix, hidden_size, num_layers):
        super(self).__init__()
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(
            weights_matrix, True)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(embedding_dim, hidden_size,
                          num_layers, batch_first=True)

    def forward(self, inp, hidden):
        return self.gru(self.embedding(inp), hidden)

    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size))